# YouTube Crawler V 1.1

- 채널명, 구독자 수 컬럼 추가
- 데이터 클렌징(value 값 한글 제거)

__-- What's next?__

- 구독자 수 단위 통일

    - 만명, 천명, 명 -> 명 단위로 통일 후 한글 제거


In [ ]:
#from selenium.webdriver.common.by import By # 언제까지 브라우저 접속을 기다려줄 지 결정
#from selenium.webdriver.support.ui import WebDriverWait # By 모듈과 같이 자주 쓰인다.
#from selenium.webdriver.support import expected_conditions as EC # 어떤 상태까지 기다리겠다(페이지 클릭이나 커서 이동)
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
from selenium import webdriver
import pandas as pd
from tqdm import tqdm
from tqdm import trange
from urllib import parse
import requests
import lxml
import time

# 검색어 입력

In [ ]:
name_txt = input('검색어 : ')
name_txt

# 크롬 웹드라이버 실행 

In [ ]:
chrome_options = Options()
#chrome_options.add_argument("--headless") # 브라우저가 화면에 뜨지 않고 내부적으로 실행

# webdriver 설정
browser = webdriver.Chrome('./chromedriver', options = chrome_options)

# 크롬 브라우저 내부 대기
#browser.implicitly_wait(6)

# chrome을 전체화면으로 넓히는 옵션
browser.maximize_window()

# 특정 URL로 브라우저 이동
youtube = 'https://www.youtube.com/'
browser.get(youtube)

# 검색창에 검색어 입력

In [ ]:
element = browser.find_element_by_name("search_query") # 검색창 클릭(객체 지정)
element.send_keys(name_txt) # 검색어 입력
element.submit() # 입력 보내기

# 필터 설정

In [ ]:
# 필터 기준 - 업로드 날짜 - 한 달
browser.find_element_by_xpath('/html/body/ytd-app/div/ytd-page-manager/ytd-search/div[1]/ytd-two-column-search-results-renderer/div/ytd-section-list-renderer/div[1]/div[2]/ytd-search-sub-menu-renderer/div[1]/div/ytd-toggle-button-renderer/a/paper-button').click() # 필터 한 달
click_1 = browser.find_element_by_xpath('/html/body/ytd-app/div/ytd-page-manager/ytd-search/div[1]/ytd-two-column-search-results-renderer/div/ytd-section-list-renderer/div[1]/div[2]/ytd-search-sub-menu-renderer/div[1]/iron-collapse/div/ytd-search-filter-group-renderer[1]/ytd-search-filter-renderer[4]/a/div/yt-formatted-string')
time.sleep(0.5)
click_1.click()

time.sleep(2)

# 필터 기준 - 정렬 기준 - 조회수
browser.find_element_by_xpath('/html/body/ytd-app/div/ytd-page-manager/ytd-search/div[1]/ytd-two-column-search-results-renderer/div/ytd-section-list-renderer/div[1]/div[2]/ytd-search-sub-menu-renderer/div[1]/div/ytd-toggle-button-renderer/a/paper-button').click()
click_2 = browser.find_element_by_xpath('/html/body/ytd-app/div/ytd-page-manager/ytd-search/div[1]/ytd-two-column-search-results-renderer/div/ytd-section-list-renderer/div[1]/div[2]/ytd-search-sub-menu-renderer/div[1]/iron-collapse/div/ytd-search-filter-group-renderer[5]/ytd-search-filter-renderer[3]/a/div/yt-formatted-string')
time.sleep(0.5)
click_2.click()

# 무한 스크롤 

In [ ]:
scroll_pane_height = browser.execute_script('return document.documentElement.scrollHeight')

while True:
    # 스크롤바를 스크롤 패인 높이만큼 이동
    browser.execute_script('window.scrollTo(0,document.documentElement.scrollHeight)')
    
    # 밑에 붙일 내용을 요청해서 화면을 만들때까지 휴지
    time.sleep(1)
    new_scroll_pane_height = browser.execute_script('return document.documentElement.scrollHeight')
    print(scroll_pane_height, new_scroll_pane_height)
    if scroll_pane_height == new_scroll_pane_height:
        break
    scroll_pane_height = new_scroll_pane_height

# 동영상 URL 가져오기

In [ ]:
# browser 내용 보기
title, url, tmp = [], [], []
soup = BeautifulSoup(browser.page_source, 'lxml')
video_url = soup.select('a#video-title')
for video in video_url:
    tmp.append(video.attrs['aria-label'].split()[-1:]) # 조회수
    url.append(youtube + video.attrs['href']) # URL
    title.append(video.attrs['title'])
URL = pd.DataFrame({'제목' : title, 'URL' : url, '조회수' : tmp})

In [ ]:
URL.sample(10)

In [ ]:
URL

In [ ]:
URL['URL']

# URL로 순서대로 접근하여 필요 정보 수집
- 업로드 시기, 조회수, 좋아요, 싫어요, 댓글수

In [ ]:
import pandas as pd
import requests
import time
import re

In [ ]:
video = browser.page_source
s = BeautifulSoup(video, 'lxml')
s

In [ ]:
channel_sub = s.find_all('div', id = 'upload-info')
channel_sub[1].select('yt-formatted-string')[1].get_text()

In [ ]:
total = pd.DataFrame()
for idx in range(len(URL['URL'])):
    if idx % 5 == 0:    print(idx)
    browser.get(URL['URL'][idx])
    time.sleep(1)
    browser.find_element_by_tag_name('body').send_keys(Keys.END)
    time.sleep(0.5)
    browser.find_element_by_tag_name('body').send_keys(Keys.END)
    time.sleep(5)
    video = browser.page_source
    s = BeautifulSoup(video, 'lxml')

    view_date = s.find('div', id = 'info-text')
    good_bad = s.find_all('div', id = 'top-level-buttons')
    channel_sub = s.find_all('div', id = 'upload-info')
    reply = s.find_all("yt-formatted-string", class_ = 'count-text style-scope ytd-comments-header-renderer')

    pattern_view = re.compile('조회수 .*회')
    pattern_date = re.compile('[0-9]+. [0-9]+. [0-9]+.')
    pattern_good = re.compile('좋아요 .*개')
    pattern_bad = re.compile('싫어요 .*개')
    pattern_reply = re.compile('댓글.*개')
    
    view_ = pattern_view.findall(str(view_date))
    view_ = str([view_[0].split('</span>')[0]])
    date_ = pattern_date.findall(str(view_date))
    good_ = str(pattern_good.findall(str(good_bad)))
    bad_ = str(pattern_bad.findall(str(good_bad)))
    reply_ = str(pattern_reply.findall(str(reply)))
    try:
        channel_ = channel_sub[1].select('a')[0].get_text()
        subscriber_ = channel_sub[1].select('yt-formatted-string')[1].get_text()[4:]
    except:
        channel_ = channel_sub[0].select('a')[0].get_text()
        subscriber_ = channel_sub[0].select('yt-formatted-string')[1].get_text()[4:]
    
    if view_ == []: 
        print('view_')
        view_ = ['NaN']
    else: view_ = view_[6:-3]
    if date_ == []: 
        print('date_')
        date_ = ['NaN']
    if good_ == []: 
        print('good_')
        good_ = ['NaN']
    else: good_ = good_[6:-3]
    if bad_ == []: 
        print('bad_')
        bad_ = ['NaN']
    else: bad_ = bad_[6:-3]
    if reply_ == []: 
        print('reply_')
        reply_ = ['NaN']
    else: reply_ = reply_[5:-3]
    if channel_ == []: 
        print('channel_')
        channel_ = ['NaN']
    if subscriber_ == []: 
        print('subscriber_')
        subscriber_ = ['NaN']
        
    df = pd.DataFrame({'제목' : [URL['제목'][idx]],'채널명' : channel_ , '구독자 수' : subscriber_, '조회수' : view_, '좋아요' : good_, '싫어요' : bad_, '댓글수' : reply_ , '업로드 날짜' : date_, 'URL' : [URL['URL'][idx]]})
    total = total.append(df)
    if idx == 5:
        break

In [ ]:
df

In [ ]:
total = total.reset_index(drop = True)
total#['댓글수']

In [ ]:
browser.close()

--- 

In [ ]:
browser.get(URL['URL'][0])

In [ ]:
browser.execute_script('window.scrollTo(0,document.documentElement.scrollHeight)')

In [ ]:
view = pattern_view.findall(str(view_date))
view = [view[0].split('</span>')[0]]
view

In [ ]:
date = pattern_date.findall(str(view_date))
date

In [ ]:
good_bad = s.find_all('div', id = 'top-level-buttons')

In [ ]:
pattern_good = re.compile('좋아요 .*개')
pattern_bad = re.compile('싫어요 .*개')

In [ ]:
pattern_good.findall(str(good_bad))

In [ ]:
pattern_bad.findall(str(good_bad))

In [ ]:
reply = s.find_all("yt-formatted-string", class_ = 'count-text style-scope ytd-comments-header-renderer')
reply

In [ ]:
pattern_reply = re.compile('댓글.*개')
pattern_reply.findall(str(reply))

In [ ]:
view_date = s.find('div', id = 'info-text')
view_date

In [ ]:
pattern_date = re.compile('[0-9]+. [0-9]+. [0-9]+.')

In [ ]:
pattern_date.findall(str(view_date))

In [ ]:
total = pd.DataFrame()
for idx in range(len(URL['URL'])):
    if idx % 5 == 0:    print(idx)
    browser.get(URL['URL'][idx])
    time.sleep(1)
    browser.find_element_by_tag_name('body').send_keys(Keys.END)
    time.sleep(0.5)
    browser.find_element_by_tag_name('body').send_keys(Keys.END)
    time.sleep(5)
    video = browser.page_source
    s = BeautifulSoup(video, 'lxml')

    view_date = s.find('div', id = 'info-text')
    good_bad = s.find_all('div', id = 'top-level-buttons')
    channel_sub = s.find_all('div', id = 'upload-info')
    reply = s.find_all("yt-formatted-string", class_ = 'count-text style-scope ytd-comments-header-renderer')

    pattern_view = re.compile('조회수 .*회')
    pattern_date = re.compile('[0-9]+. [0-9]+. [0-9]+.')
    pattern_good = re.compile('좋아요 .*개')
    pattern_bad = re.compile('싫어요 .*개')
    pattern_reply = re.compile('댓글.*개')
    
    view_ = pattern_view.findall(str(view_date))
    view_ = str([view_[0].split('</span>')[0]])
    date_ = pattern_date.findall(str(view_date))
    good_ = str(pattern_good.findall(str(good_bad)))
    bad_ = str(pattern_bad.findall(str(good_bad)))
    reply_ = str(pattern_reply.findall(str(reply)))
    try:
        channel_ = channel_sub[1].select('a')[0].get_text()
        subscriber_ = channel_sub[1].select('yt-formatted-string')[1].get_text()[4:]
    except:
        channel_ = channel_sub[0].select('a')[0].get_text()
        subscriber_ = channel_sub[0].select('yt-formatted-string')[1].get_text()[4:]
    
    if view_ == []: 
        print('view_')
        view_ = ['NaN']
    else: view_ = view_[6:-3]
    if date_ == []: 
        print('date_')
        date_ = ['NaN']
    if good_ == []: 
        print('good_')
        good_ = ['NaN']
    else: good_ = good_[6:-3]
    if bad_ == []: 
        print('bad_')
        bad_ = ['NaN']
    else: bad_ = bad_[6:-3]
    if reply_ == []: 
        print('reply_')
        reply_ = ['NaN']
    else: reply_ = reply_[5:-3]
    if channel_ == []: 
        print('channel_')
        channel_ = ['NaN']
    if subscriber_ == []: 
        print('subscriber_')
        subscriber_ = ['NaN']
        
    df = pd.DataFrame({'제목' : [URL['제목'][idx]],'채널명' : channel_ , '구독자 수' : subscriber_, '조회수' : view_, '좋아요' : good_, '싫어요' : bad_, '댓글수' : reply_ , '업로드 날짜' : date_, 'URL' : [URL['URL'][idx]]})
    total = total.append(df)
    if idx == 5:
        break